In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 필요함수 호출

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import geopy.distance # 거리계산 패키지
from collections import Counter # count 용도
from pycaret.regression import * #최적의 모델찾기

## train, test데이터 불러오기

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Lecture/jeju/train.csv')
test=pd.read_csv('/content/drive/MyDrive/Lecture/jeju/test.csv')

In [ ]:
# 해당 주요 장소의 임의 지역 위도, 경도

jeju=(33.51411, 126.52969) # 제주 측정소 근처
gosan=(33.29382, 126.16283) #고산 측정소 근처
seongsan=(33.38677, 126.8802) #성산 측정소 근처
po=(33.24616, 126.5653) #서귀포 측정소 근처

In [ ]:
#정류장의 위치만 확인하기 위해 groupby를 실행함
data=train[['latitude','longitude','station_name']].drop_duplicates(keep='first')

data2=data.groupby(['station_name'])['latitude','longitude'].mean()

data2.to_csv("folium.csv")

data2=pd.read_csv("folium.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [ ]:
t1 = [geopy.distance.vincenty( (i,j), jeju).km for i,j in list( zip( train['latitude'],train['longitude'] )) ]
t2 = [geopy.distance.vincenty( (i,j), gosan).km for i,j in list( zip( train['latitude'],train['longitude'] )) ]
t3 = [geopy.distance.vincenty( (i,j), seongsan).km for i,j in list( zip( train['latitude'],train['longitude'] )) ]
t4 = [geopy.distance.vincenty( (i,j), po).km for i,j in list( zip( train['latitude'],train['longitude'] )) ]

train['dis_jeju']=t1
train['dis_gosan']=t2
train['dis_seongsan']=t3
train['dis_po']=t4

In [ ]:
# 각측정소와 정류소의 거리를 계산한 다음, 해당 정류소에서
# 가장 가까운 곳에 있는 측정소를 dist_name변수의 값으로 넣는다.
total=pd.DataFrame( list(zip( t1,t2,t3,t4)),columns=['jeju','gosan','seongsan','po'] )
train['dist_name'] = total.apply(lambda x: x.argmin(), axis=1)

data22=train[['station_name','latitude','longitude','dist_name']].drop_duplicates(keep='first')

Counter(data22['dist_name'])

Counter({0: 1479, 1: 659, 2: 597, 3: 828})

In [ ]:
t1 = [geopy.distance.vincenty( (i,j), jeju).km for i,j in list( zip( test['latitude'],test['longitude'] )) ]
t2 = [geopy.distance.vincenty( (i,j), gosan).km for i,j in list( zip( test['latitude'],test['longitude'] )) ]
t3 = [geopy.distance.vincenty( (i,j), seongsan).km for i,j in list( zip( test['latitude'],test['longitude'] )) ]
t4 = [geopy.distance.vincenty( (i,j), po).km for i,j in list( zip( test['latitude'],test['longitude'] )) ]

test['dis_jeju']=t1
test['dis_gosan']=t2
test['dis_seongsan']=t3
test['dis_po']=t4

total=pd.DataFrame( list(zip( t1,t2,t3,t4)),columns=['jeju','gosan','seongsan','po'] )
test['dist_name'] = total.apply(lambda x: x.argmin(), axis=1)

#날씨데이터 추가

In [ ]:
weather = pd.read_csv("/content/drive/MyDrive/Lecture/jeju/train_weather.csv",encoding='cp949')
test_weather = pd.read_csv("/content/drive/MyDrive/Lecture/jeju/test_weather.csv",encoding='cp949')

Na값은 비가 안오는 날을 뜻하는 것이기 때문에 0으로 채워준다.

In [ ]:
weather = weather.fillna(0)
test_weather = test_weather.fillna(0)

비가 오면 1 안오면 0

In [ ]:
def f(x):
  if x == 0:
    return 0
  else:
    return 1

In [ ]:
weather['rain'] = weather['강수량(mm)'].apply(f)
test_weather['rain'] = test_weather['강수량(mm)'].apply(f)

Train과 test row이름과 같게 만들어줌

In [ ]:
weather = weather.rename(columns = {"일시":"date","지점":"dist_name"})
test_weather = test_weather.rename(columns = {"일시":"date","지점":"dist_name"})

In [ ]:
weather_1 = weather[['dist_name','지점명','date','rain']]
test_weather = test_weather[['dist_name','지점명','date','rain']]

dist name 생성

In [ ]:
def f(x):
  if x == 184:
    return 0
  if x == 185:
    return 1
  if x == 188:
    return 2
  else:
    return 3

In [ ]:
weather_1['dist_name'] = weather_1['dist_name'].apply(f)
test_weather['dist_name'] = test_weather['dist_name'].apply(f)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


매일 1시 rain을 가져옴

In [ ]:
numbers = range(0,2880,24)
data_rain = []
for x in numbers:
  df = weather_1.loc[x]
  data_rain.append(df)

In [ ]:
df = pd.DataFrame(data_rain)

,dist_name,지점명,date,rain
0,0,제주,2019-09-01 00:00,0
24,0,제주,2019-09-02 00:00,1
48,0,제주,2019-09-03 00:00,0
72,0,제주,2019-09-04 00:00,0
96,0,제주,2019-09-05 00:00,0
...,...,...,...,...
2760,3,서귀포,2019-09-26 00:00,0
2784,3,서귀포,2019-09-27 00:00,0
2808,3,서귀포,2019-09-28 00:00,0
2832,3,서귀포,2019-09-29 00:00,0


In [ ]:
numbers = range(0,1536,24)
test_rain = []
for x in numbers:
  df_1 = test_weather.loc[x]
  test_rain.append(df_1)

In [ ]:
test_df = pd.DataFrame(test_rain)

,dist_name,지점명,date,rain
0,0,제주,2019-10-01 00:00,0
24,0,제주,2019-10-02 00:00,1
48,0,제주,2019-10-03 00:00,1
72,0,제주,2019-10-04 00:00,0
96,0,제주,2019-10-05 00:00,0
...,...,...,...,...
1416,3,서귀포,2019-10-12 00:00,0
1440,3,서귀포,2019-10-13 00:00,0
1464,3,서귀포,2019-10-14 00:00,0
1488,3,서귀포,2019-10-15 00:00,0


필요없어서 지워줌

In [ ]:
df = df.drop(columns = ['지점명'])
test_df = test_df.drop(columns= ['지점명'])

날짜만 나오도록 시간 없애줌

In [ ]:
df_1 = pd.DataFrame({'date_1':df['date']})
df_1['date_1'] = pd.to_datetime(df_1['date_1']).dt.date
test_weather_1 = pd.DataFrame({'date_1':test_df['date']})
test_weather_1['date_1'] = pd.to_datetime(test_weather_1['date_1']).dt.date

In [ ]:
df = pd.concat([df, df_1], axis =1 )

,dist_name,date,rain,date_1
0,0,2019-09-01 00:00,0,2019-09-01
24,0,2019-09-02 00:00,1,2019-09-02
48,0,2019-09-03 00:00,0,2019-09-03
72,0,2019-09-04 00:00,0,2019-09-04
96,0,2019-09-05 00:00,0,2019-09-05
...,...,...,...,...
2760,3,2019-09-26 00:00,0,2019-09-26
2784,3,2019-09-27 00:00,0,2019-09-27
2808,3,2019-09-28 00:00,0,2019-09-28
2832,3,2019-09-29 00:00,0,2019-09-29


In [ ]:
test_weather = pd.concat([test_df, test_weather_1], axis=1)

,dist_name,date,rain,date_1
0,0,2019-10-01 00:00,0,2019-10-01
24,0,2019-10-02 00:00,1,2019-10-02
48,0,2019-10-03 00:00,1,2019-10-03
72,0,2019-10-04 00:00,0,2019-10-04
96,0,2019-10-05 00:00,0,2019-10-05


In [ ]:
df = df.drop(columns = ['date'])
test_weather = test_weather.drop(columns= ['date'])

In [ ]:
df['date'] = pd.to_datetime(df['date_1'])
test_weather['date'] = pd.to_datetime(test_weather['date_1'])

merge를 실행하기 위해 type 변경

In [ ]:
df.date=df.date.astype(str) #trian과 df의 date dtype이 달라서 merge실행이 안됨
df.dist_name =df.dist_name.astype(str)
#train.dist_name = train.dist_name.astype(str)
test_weather.dist_name = test_weather.dist_name.astype(str)
test_weather.date = test_weather.date.astype(str)

In [ ]:
df = df.drop(columns = ['date_1'])
test_weather = test_weather.drop(columns= ['date_1'])
#test_weather = test_weather.drop(columns=['지점명'])

merge 실행

In [ ]:
df_weather = pd.merge(train, df, left_on = 'date', right_on = 'date' ,how = 'outer') #outer join
df_weather2 = pd.merge(test, test_weather, left_on = 'date', right_on = 'date', how = 'outer')

dist_nmae이 같은 것끼리만 남겨줌

In [ ]:
df_weather = df_weather.dropna()

In [ ]:
df_weather['dist_name_y'] = df_weather['dist_name_y'].astype(str).astype(int)

In [ ]:
train = df_weather.loc[df_weather['dist_name_x'] == df_weather['dist_name_y']]

In [ ]:
df_weather2 = df_weather2.dropna()

In [ ]:
df_weather2['dist_name_y'] = df_weather2['dist_name_y'].astype(str).astype(int)

In [ ]:
test = df_weather2.loc[df_weather2['dist_name_x']== df_weather2['dist_name_y']]

## train과 test데이터의 시작일, 끝일 확인

In [ ]:
train['date'].min(),train['date'].max()

('2019-09-01', '2019-09-30')

In [ ]:
test['date'].min(), test['date'].max()

('2019-10-01', '2019-10-16')

## 데이터 결측값 확인  - 결측값 없음

In [ ]:
train.isnull().sum()

id               0
date             0
bus_route_id     0
in_out           0
station_code     0
station_name     0
latitude         0
longitude        0
6~7_ride         0
7~8_ride         0
8~9_ride         0
9~10_ride        0
10~11_ride       0
11~12_ride       0
6~7_takeoff      0
7~8_takeoff      0
8~9_takeoff      0
9~10_takeoff     0
10~11_takeoff    0
11~12_takeoff    0
18~20_ride       0
dis_jeju         0
dis_gosan        0
dis_seongsan     0
dis_po           0
dist_name_x      0
dist_name_y      0
rain             0
dtype: int64

## date를 날짜형식으로 바꾸기 - 요일별로 바꾸기 위해 사용

In [ ]:
train['date'] = pd.to_datetime(train['date'])
test['date']=pd.to_datetime(test['date'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train['date'].head(3)

0   2019-09-01
4   2019-09-01
8   2019-09-01
Name: date, dtype: datetime64[ns]

## date를 요일로 나누기
## 9월달에는 추석, 10월달에는 한글날,개천절 열을 쉬는날로 빼기

In [ ]:
train['weekday'] = train['date'].dt.weekday
test['weekday']=test['date'].dt.weekday
train['weekday'][156277:176360]=7
test['weekday'][28632:40327]=7
test['weekday'][112356:125267]=7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

## 월화수목금토일 공휴일 순서

In [ ]:
set(train['weekday'])

{0, 1, 2, 3, 4, 5, 6, 7}

## 평일 : 0 쉬는날 : 1로 변환하기
### 쉬는날=공휴일, 주말
### 0,1,2,3,4 = 0       -        5,6,7 =1

In [ ]:
train['weekday']=train['weekday'].map({0:0,1:0,2:0,3:0,4:0,5:1,6:1,7:1})
test['weekday']=test['weekday'].map({0:0,1:0,2:0,3:0,4:0,5:1,6:1,7:1})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
set(train['weekday']),set(test['weekday'])

({0, 1}, {0, 1})

## 시내버스 : 0 시외버스 : 1로 변환하기

In [ ]:
train['in_out'] = train['in_out'].map({'시내':0,'시외':1})
test['in_out'] = test['in_out'].map({'시내':0,'시외':1})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
set(train['in_out'])

{0, 1}

In [ ]:
plt.hist(train['in_out'],bins=2)

## target변수 (18~20ride)와 맞게 시간을 2시간씩 묶기

In [ ]:
train['6~8_ride']=train['6~7_ride']+train['7~8_ride']
train['8~10_ride']=train['8~9_ride']+train['9~10_ride']
train['10~12_ride']=train['10~11_ride']+train['11~12_ride']


test['6~8_ride']=test['6~7_ride']+test['7~8_ride']
test['8~10_ride']=test['8~9_ride']+test['9~10_ride']
test['10~12_ride']=test['10~11_ride']+test['11~12_ride']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

## 더이상 사용하지 않을 변수 삭제하기

#### id - index만 나타내기 때문에 삭제
#### station_name - 카테고리변수로 넣어봤지만 효과 없음
#### take_off - 0값이 너무 많아 확인해본 결과 하차태그를 안찍으면 0값이 나옴
####                  분석에 방해가 되서 삭제
#### ride - 2시간씩 묶은 변수를 사용

In [ ]:
train_date=train.drop(['station_name','date','id','6~7_ride','7~8_ride','8~9_ride','9~10_ride','10~11_ride','11~12_ride','6~7_takeoff','7~8_takeoff','8~9_takeoff','9~10_takeoff','10~11_takeoff','11~12_takeoff'],axis=1)
test_date=test.drop(['station_name','date','id','6~7_ride','7~8_ride','8~9_ride','9~10_ride','10~11_ride','11~12_ride','6~7_takeoff','7~8_takeoff','8~9_takeoff','9~10_takeoff','10~11_takeoff','11~12_takeoff'],axis=1)

In [ ]:
train_final=train_date.drop(['dist_name_y'],axis=1)
test_final=test_date.drop(['dist_name_y'],axis=1)

In [ ]:
train_final.head()

## 최종 사용변수
#### bus_route_id - 위경도별로 묶어놓은 값이라 중요한 역할을함
#### in_out - 시내, 시외버스별로 나눠서 사용하면 좋음
#### station_code - 정류장별로 나눠있어서 사용하면 좋음
#### lon,lat - 위도, 경도에 따라 중요한 변수로 사용가능
#### weekday - 쉬는날과 쉬지않는날로 부분해서 중요한 역할을 함
#### ride - target에 맞게 2시간씩 묶어서 사용

## Pycaret을 통해 최적의 모델 찾기

In [ ]:
data = setup(data = train_final, target ='18~20_ride',use_gpu=True)

In [ ]:
##사용하는 평가지표 이름을 넣으면 해당 평가지표 순서대로 모델 순위 보여줌
model = compare_models(sort='RMSE')

## x_train= train에서 예측값을 제외하고
## x_label=train에서 target변수만 남기기

In [ ]:
x_train=train_date.drop(['18~20_ride'],axis=1)
x_label=train_date['18~20_ride']

### sklearn.model_selection - train_test_split사용
### train과 val로 나누기 8:2비율

In [ ]:
x_train,y_train=train_test_split(x_train,test_size=0.2,random_state=42)
x_labels,y_labels=train_test_split(x_label,test_size=0.2,random_state=42)

## Catboost모델 사용

In [ ]:
from catboost import CatBoostRegressor

## Gridsearch를 통한 최적의 파라미터 찾기

In [ ]:
cat_bst=CatBoostRegressor()
param_dist = {"max_depth": [8,10,12],
              "learning_rate" : [0.1],
              "n_estimators": [3000,5000,7000]
             }
grid_search = GridSearchCV(cat_bst, n_jobs=-1, param_grid=param_dist, cv = 3, verbose=5)
grid_search.fit(x_train,x_label)

In [ ]:
grid_search.best_params_

## 찾아낸 파라미터로 모델학습시키기

In [ ]:
params = {"learning_rate":0.1,
                          "n_estimators":3500,
                          "depth":10}
cat_bst=CatBoostRegressor(**params,random_state=42)
cat_bst.fit(x_train,x_labels,eval_set=(y_train,y_labels))

0:	learn: 4.5529307	test: 4.2550923	best: 4.2550923 (0)	total: 133ms	remaining: 7m 46s
1:	learn: 4.3505486	test: 4.0737458	best: 4.0737458 (1)	total: 259ms	remaining: 7m 33s
2:	learn: 4.1657908	test: 3.9099842	best: 3.9099842 (2)	total: 389ms	remaining: 7m 33s
3:	learn: 4.0035178	test: 3.7652611	best: 3.7652611 (3)	total: 514ms	remaining: 7m 29s
4:	learn: 3.8662090	test: 3.6463004	best: 3.6463004 (4)	total: 650ms	remaining: 7m 34s
5:	learn: 3.7423959	test: 3.5435690	best: 3.5435690 (5)	total: 783ms	remaining: 7m 35s
6:	learn: 3.6374411	test: 3.4535087	best: 3.4535087 (6)	total: 924ms	remaining: 7m 41s
7:	learn: 3.5415470	test: 3.3692127	best: 3.3692127 (7)	total: 1.05s	remaining: 7m 38s
8:	learn: 3.4629225	test: 3.3027309	best: 3.3027309 (8)	total: 1.18s	remaining: 7m 38s
9:	learn: 3.3923144	test: 3.2459806	best: 3.2459806 (9)	total: 1.32s	remaining: 7m 40s
10:	learn: 3.3134580	test: 3.1779672	best: 3.1779672 (10)	total: 1.45s	remaining: 7m 41s
11:	learn: 3.2553698	test: 3.1274627	best

KeyboardInterrupt: ignored

### 특성의 중요도

In [ ]:
# Plot feature importance
feature_importance = cat_bst.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5

plt.figure(figsize=(6,6))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, x_train.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

### 실제값과 에측값 차이 비교하기

In [ ]:
#실제값
x= train_date['18~20_ride']
#예측값
y= cat_bst.predict(x_train)

In [ ]:
plt.plot(x,c='red')
plt.plot(y,c='blue')
# 범위
plt.xlim(0, 10)
plt.ylim(0, 55)
plt.legend(['original','predict' ])
plt.title('Compare original predict')


## predict값을 test의 target값에 넣기

In [ ]:
test['18~20_ride']=model_lgb.predict(test_date)

## 결과를 csv파일로 변환후 제출

In [ ]:
test[['id','18~20_ride']].to_csv("jhlgb_score.csv",index=False)